In [1]:
# This is a solution script for workflow #1 using results from 3(+) reasoning teams
#### Set the input disease, change this value to test a new disease
input_disease = "DOID:9352"


In [2]:
#### Import some needed modules
import requests
import json
#### Workflow 1

num_robocop_results = 25
output_format="DENSE"

#### Set the base URL for the reasoner and its endpoint
XRAY_API_BASE_URL = 'https://rtx.ncats.io/api/rtx/v1'
xray_url_str = XRAY_API_BASE_URL + "/query"

ROBOCOP_API_BASE_URL = 'http://robokop.renci.org/api/'
robocop_mod3_url_str = ROBOCOP_API_BASE_URL + "simple/quick/template/wf1mod3/%s/?max_results=%d&output_format=%s" % (input_disease, num_robocop_results,output_format)
robocop_mod3a_url_str = ROBOCOP_API_BASE_URL + "simple/quick/template/wf1mod3_v2/%s/?max_results=%d&output_format=%s" % (input_disease, num_robocop_results,output_format)

annot_url_str = "https://rtx.ncats.io/api/rtx/v1/response/process"

In [3]:
################################################################
# X-ray module 1: given a disease, find genetic conditions that share "representative phenotypes" in common

#### Create a dict of the request, specifying the query type and its parameters
request = {"query_type_id": "Q10001", "terms": {"disease": input_disease}}

#### Send the request to RTX and check the status
response_content = requests.post(xray_url_str, json=request, headers={'accept': 'application/json'})
status_code = response_content.status_code
assert status_code == 200 !=0, "Module 1 failed"
module1_xray_results_json = response_content.json()
print('Module 1 completed successfully. ' + str(len(module1_xray_results_json)) + ' conditions with similar phenotypes found\n')
for result in module1_xray_results_json['result_list']:
        print(result['essence'])

Module 1 completed successfully. 16 conditions with similar phenotypes found

HEART DEFECTS, CONGENITAL, AND OTHER CONGENITAL ANOMALIES; HDCA
DIABETES MELLITUS, PERMANENT NEONATAL; PNDM
PANCREATIC AND CEREBELLAR AGENESIS; PACA
MATURITY-ONSET DIABETES OF THE YOUNG; MODY
RENAL CYSTS AND DIABETES SYNDROME; RCAD
LIPODYSTROPHY, FAMILIAL PARTIAL, TYPE 6; FPLD6
LIPODYSTROPHY, CONGENITAL GENERALIZED, TYPE 1; CGL1
LIPODYSTROPHY, CONGENITAL GENERALIZED, TYPE 2; CGL2
LIPODYSTROPHY, FAMILIAL PARTIAL, TYPE 2; FPLD2
MANDIBULOACRAL DYSPLASIA WITH TYPE A LIPODYSTROPHY; MADA
LIPODYSTROPHY, FAMILIAL PARTIAL, TYPE 5; FPLD5
HYPOTHYROIDISM, CENTRAL, AND TESTICULAR ENLARGEMENT; CHTE
HYPOBETALIPOPROTEINEMIA, FAMILIAL, 2; FHBL2
MATURITY-ONSET DIABETES OF THE YOUNG, TYPE 8, WITH EXOCRINE DYSFUNCTION; MODY8
MATURITY-ONSET DIABETES OF THE YOUNG, TYPE 11; MODY11
APOLIPOPROTEIN C-III DEFICIENCY
OSTEOPETROSIS, AUTOSOMAL RECESSIVE 6; OPTB6
DIABETES MELLITUS, TRANSIENT NEONATAL, 3
GLOMERULOCYSTIC KIDNEY DISEASE WITH 

In [4]:
request = {"query_type_id": "Q55", "terms": {"disease": input_disease}}

#### Send the request to RTX and check the status
response_content = requests.post(xray_url_str, json=request, headers={'accept': 'application/json'})
status_code = response_content.status_code
assert status_code == 200 !=0, "Module 2 failed"
module2_xray_results_json = response_content.json()

myResultSet = set()
for result in module2_xray_results_json['result_list']:
    if 'row_data' in result:
        myResultSet.add((result['row_data'][2],result['row_data'][3]))
        
#this is to resolve potential caching issues that cause results without row_data to be returned
#Could use some cleanup to resolve the repetition
if(len(myResultSet)==0):
    request = {"query_type_id": "Q55", "terms": {"disease": input_disease}, "bypass_cache": "true"}
    response_content = requests.post(xray_url_str, json=request, headers={'accept': 'application/json'})
    status_code = response_content.status_code
    assert status_code == 200 !=0, "Module 2 failed"
    module2_xray_results_json = response_content.json()
    
    for result in module2_xray_results_json['result_list']:
        if 'row_data' in result:
            myResultSet.add((result['row_data'][2],result['row_data'][3]))

print("Module 2 completed successfully")
print(str(len(myResultSet))+' drugs found:')
for drug in myResultSet:
   print(drug)

Module 2 completed successfully
25 drugs found:
('insulin lispro', 'CHEMBL.COMPOUND:CHEMBL1201538')
('medrysone', 'CHEMBL.COMPOUND:CHEMBL1201173')
('methyltestosterone', 'CHEMBL.COMPOUND:CHEMBL1395')
('insulin aspart', 'CHEMBL.COMPOUND:CHEMBL1201496')
('insulin glulisine', 'CHEMBL.COMPOUND:CHEMBL1201613')
('calusterone', 'CHEMBL.COMPOUND:CHEMBL455706')
('methandrostenolone', 'CHEMBL.COMPOUND:CHEMBL1418176')
('norgestimate', 'CHEMBL.COMPOUND:CHEMBL1200934')
('insulin glargine', 'CHEMBL.COMPOUND:CHEMBL1201497')
('norethandrolone', 'CHEMBL.COMPOUND:CHEMBL1697845')
('insulin human', 'CHEMBL.COMPOUND:CHEMBL1201631')
('repaglinide', 'CHEMBL.COMPOUND:CHEMBL1272')
('sitagliptin', 'CHEMBL.COMPOUND:CHEMBL1422')
('etonogestrel', 'CHEMBL.COMPOUND:CHEMBL1531')
('canrenone', 'CHEMBL.COMPOUND:CHEMBL1463345')
('norelgestromin', 'CHEMBL.COMPOUND:CHEMBL1200807')
('mifepristone', 'CHEMBL.COMPOUND:CHEMBL1276308')
('insulin degludec', 'CHEMBL.COMPOUND:CHEMBL2107869')
('insulin pork', 'CHEMBL.COMPOUND:CHEMB

In [5]:
################################################################
# Gamma team module 3: agent-centric
# Mod 3 un-lettered approach
response_content = requests.get(robocop_mod3_url_str, json={}, headers={'accept': 'application/json'})
status_code = response_content.status_code
assert status_code == 200 !=0, "Module 3 failed"
module3_robocop_results_json = response_content.json()
print("Module 3 completed successfully")
try:
    print(str(len(module3_robocop_results_json['result_list']))+' related chemical substances found')
except:
    print ("Module 3 found no related chemical substances")

# Mod3a approach
response_content = requests.get(robocop_mod3a_url_str, json={}, headers={'accept': 'application/json'})
status_code = response_content.status_code
assert status_code == 200 !=0 ,"Module 3a failed"
module3a_robocop_results_json = response_content.json()
print("Module 3a completed successfully")
try:
    print(str(len(module3a_robocop_results_json['result_list']))+' related chemical substances found')
except: 
    print("Module 3a found no related chemical substances")


Module 3 completed successfully
25 related chemical substances found
Module 3a completed successfully
25 related chemical substances found


In [22]:
################################################################
# Orange team module 4+5: annotation and scoring

# annotate the x-ray results
to_post = {"options": ["AnnotateDrugs", "Store", "ReturnResponseId"], "responses": [module2_xray_results_json]}
module2_xray_results_annot_json = requests.post(annot_url_str, json=to_post)
# annotate gamma 3
to_post = {"options": ["AnnotateDrugs", "Store", "ReturnResponseId"], "responses": [module3_robocop_results_json]}
module3_robocop_results_annot_json = requests.post(annot_url_str, json=to_post)
#annotate gamma 3a
to_post = {"options": ["AnnotateDrugs", "Store", "ReturnResponseId"], "responses": [module3a_robocop_results_json]}
module3a_robocop_results_annot_json = requests.post(annot_url_str, json=to_post)

all = module2_xray_results_annot_json
# merge results
if module3_robocop_results_annot_json.json()['status']==200:
    to_post = {"options": ["AnnotateDrugs", "Store", "ReturnResponseId"], "responseURIs":[module2_xray_results_json['id']],"responses": [module3_robocop_results_json]}
    all = requests.post(annot_url_str, json=to_post)
    
if module3a_robocop_results_annot_json.json()['status']==200:
    to_post = {"options": ["AnnotateDrugs", "Store", "ReturnResponseId"], "responseURIs":[module2_xray_results_json['id']],"responses": [module3a_robocop_results_json]}
    all = requests.post(annot_url_str, json=to_post)

print("Annotation completed")


Annotation completed


In [23]:
################################################################
# Visualization
# The above API call creates a website (dynamically) where the results can be viewed
print("Please visit the following website: https://rtx.ncats.io/devLM/list.html?r=%s" % all.json()['response_id'])

# will return something like the following:
# https://rtx.ncats.io/devLM/list.html?r=473

Please visit the following website: https://rtx.ncats.io/devLM/list.html?r=1416
